<a href="https://colab.research.google.com/github/obenadak/NewsLDA/blob/master/news_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [4]:
df = pd.read_csv('/content/turkish_news_70000.csv')
df.head(5)

,id,main_image,published,site,text,title,url
0,72337,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:33:00.000+02:00""","""diken.com.tr""","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""http://www.diken.com.tr/yatirim-bankasi-dolar..."
1,72338,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:21:00.000+02:00""","""diken.com.tr""","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""http://www.diken.com.tr/piar-arastirma-adana-..."
2,72339,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T12:20:00.000+02:00""","""diken.com.tr""","""Renaissance Capital: Merkez Bankası bu hafta ...","""Renaissance Capital: Merkez Bankası bu hafta ...","""http://www.diken.com.tr/renaissance-capital-m..."
3,72340,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T11:53:00.000+02:00""","""diken.com.tr""","""Ağrı’daki İYİ Partililerin istifa gerekçesi: ...","""Ağrı’daki İYİ Partililerin istifa gerekçesi: ...","""http://www.diken.com.tr/agridaki-iyi-partilil..."
4,72341,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T11:52:00.000+02:00""","""diken.com.tr""","""Otomobil pazarı iki ayda yarı yarıya eridi 04...","""Otomobil pazarı iki ayda yarı yarıya eridi""","""http://www.diken.com.tr/otomobil-pazari-iki-a..."


In [5]:
haber_df = df[["text"]] #text sütunu yeterli
haber_df.head(5)

,text
0,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9..."
1,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i..."
2,"""Renaissance Capital: Merkez Bankası bu hafta ..."
3,"""Ağrı’daki İYİ Partililerin istifa gerekçesi: ..."
4,"""Otomobil pazarı iki ayda yarı yarıya eridi 04..."


In [6]:
#metin temizleme
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

punctions_cluster = string.punctuation
stopwords_cluster = stopwords.words('turkish')
stopwords_cluster.extend(["bir","kadar","sonra"])

def metin_temizleme(metin):
    metin = metin.lower()
    metin = metin.replace("\\n", " ")

    # Tüm farklı kesme işareti çeşitlerini ele alalım
    metin = re.sub("’(\w+)", "", metin)  # Tüm kesme işareti çeşitleri için
    #metin = re.sub("‘(\w+)", "", metin)

    #metin = re.sub("'(\w+)", "", metin)

    # Gereksiz karakterleri kaldırmak
    metin = re.sub("[❝,❛,❜,❞,‘,’,']", "", metin)

    # Rakamları kaldırmak
    metin = re.sub("[0-9]+", "", metin)

    # Noktalama işaretlerini temizlemek
    metin = "".join(list(map(lambda x: x if x not in punctions_cluster else " ", metin)))

    # Stop word'leri kaldırmak (örnek stopwords)
    metin = " ".join([i for i in metin.split() if i not in stopwords_cluster])

    # Tek harfli kelimeleri kaldırmak
    metin = " ".join([i for i in metin.split() if len(i) > 1])

    return metin

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
haber_df.loc[:, "cleaning_text"] = ' '

for idx, item in enumerate(haber_df["text"].values):
    cleaned_text = metin_temizleme(item)
    # Assign the cleaned text to the corresponding row in the 'cleaning_text' column
    haber_df.loc[idx, "cleaning_text"] = cleaned_text


<ipython-input-7-f2658ef7db5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_df.loc[:, "cleaning_text"] = ' '


In [8]:
haber_df["cleaning_text"]

,cleaning_text
0,yatırım bankası dolar tl üçüncü çeyrekte görec...
1,pi̇ar araştırma adana antalya millet ittifakı ...
2,renaissance capital merkez bankası hafta baz p...
3,ağrı i̇yi̇ partililerin istifa gerekçesi akp s...
4,otomobil pazarı iki ayda yarı yarıya eridi oto...
...,...
72104,prodigy solisti keith flint evinde ölü bulundu...
72105,adana sahte icra müdürü gerçeğini dolandırdı a...
72106,maçoğlu athena gökhan sözler bize enerji verdi...
72107,hasköy hızır gelmiş meçhul birisi kapılara lir...


In [9]:
haber_df.loc[:, "cleaning_text_token"] = haber_df["cleaning_text"].apply(lambda x: x.split())

<ipython-input-9-184ad84b02ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_df.loc[:, "cleaning_text_token"] = haber_df["cleaning_text"].apply(lambda x: x.split())


In [10]:
haber_df["cleaning_text_token"]

,cleaning_text_token
0,"[yatırım, bankası, dolar, tl, üçüncü, çeyrekte..."
1,"[pi̇ar, araştırma, adana, antalya, millet, itt..."
2,"[renaissance, capital, merkez, bankası, hafta,..."
3,"[ağrı, i̇yi̇, partililerin, istifa, gerekçesi,..."
4,"[otomobil, pazarı, iki, ayda, yarı, yarıya, er..."
...,...
72104,"[prodigy, solisti, keith, flint, evinde, ölü, ..."
72105,"[adana, sahte, icra, müdürü, gerçeğini, doland..."
72106,"[maçoğlu, athena, gökhan, sözler, bize, enerji..."
72107,"[hasköy, hızır, gelmiş, meçhul, birisi, kapıla..."


In [11]:
haber_df.head(3)

,text,cleaning_text,cleaning_text_token
0,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...",yatırım bankası dolar tl üçüncü çeyrekte görec...,"[yatırım, bankası, dolar, tl, üçüncü, çeyrekte..."
1,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i...",pi̇ar araştırma adana antalya millet ittifakı ...,"[pi̇ar, araştırma, adana, antalya, millet, itt..."
2,"""Renaissance Capital: Merkez Bankası bu hafta ...",renaissance capital merkez bankası hafta baz p...,"[renaissance, capital, merkez, bankası, hafta,..."


In [12]:
haber_df["cleaning_text_token"][0]

['yatırım',
 'bankası',
 'dolar',
 'tl',
 'üçüncü',
 'çeyrekte',
 'görecek',
 'toronto',
 'merkezli',
 'yatırım',
 'bankası',
 'td',
 'securities',
 'üçüncü',
 'çeyrekte',
 'liranın',
 'dolar',
 'karşısında',
 'yüzde',
 'kayıp',
 'yaşayarak',
 'göreceğini',
 'iddia',
 'etti',
 'fotoğraf',
 'reuters',
 'piyasa',
 'izleyecilerinin',
 'genel',
 'beklentisi',
 'üçüncü',
 'çeyrekte',
 'liranın',
 'yüzde',
 'altında',
 'kayıp',
 'yaşayacağı',
 'reklam',
 'bloomberg',
 'international',
 'yer',
 'alan',
 'habere',
 'göre',
 'td',
 'securities',
 'cuma',
 'günü',
 'yayınladığı',
 'raporda',
 'genel',
 'piyasa',
 'beklentilerinin',
 'aksine',
 'liranın',
 'yeni',
 'kriz',
 'yaşayacağını',
 'öngördü',
 'bankanın',
 'gelişmekte',
 'olan',
 'pazarlar',
 'strateji',
 'direktörü',
 'cristian',
 'maggio',
 'önce',
 'ilk',
 'çeyrekte',
 'lirada',
 'böyle',
 'kayıp',
 'olacağını',
 'öngörmüş',
 'fakat',
 'koşullar',
 'kendisini',
 'bunun',
 'ikinci',
 'çeyrekten',
 'önce',
 'olmayacağına',
 'karar',
 'v

In [13]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.1 MB/s eta 0:00:00


In [24]:
!pip install gensim
import gensim

tokened_text = haber_df["cleaning_text_token"]
word_list = gensim.corpora.Dictionary(tokened_text)

word_list.filter_extremes(no_below=1, no_above=0.7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
term_matrix = [word_list.doc2bow(terim) for terim in tokened_text]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = term_matrix,
                                           id2word = word_list,
                                           num_topics = 15,
                                           passes = 10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
lda_model.save('model.gensim')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
topics = lda_model.print_topics(num_words = 7)

for topic in topics:
    print(topic)

(0, '0.034*"yüzde" + 0.021*"tl" + 0.018*"bin" + 0.015*"milyon" + 0.015*"milyar" + 0.013*"dolar" + 0.012*"göre"')
(1, '0.060*"gezi" + 0.041*"hürriyet" + 0.028*"mayıs" + 0.018*"lüks" + 0.015*"co" + 0.015*"gazetesi" + 0.013*"yayın"')
(2, '0.010*"türkiye" + 0.008*"olarak" + 0.006*"yeni" + 0.005*"dedi" + 0.005*"önemli" + 0.005*"türk" + 0.005*"büyük"')
(3, '0.042*"yorum" + 0.034*"yorumlar" + 0.030*"com" + 0.027*"haberturk" + 0.017*"sayfalarında" + 0.014*"şekilde" + 0.014*"türkiye"')
(4, '0.010*"belediye" + 0.010*"chp" + 0.010*"var" + 0.009*"başkan" + 0.008*"parti" + 0.007*"adayı" + 0.007*"başkanı"')
(5, '0.012*"olay" + 0.009*"polis" + 0.007*"kişi" + 0.007*"geldi" + 0.007*"ekipleri" + 0.007*"meydana" + 0.007*"gelen"')
(6, '0.007*"kadın" + 0.007*"şubat" + 0.006*"son" + 0.006*"bölüm" + 0.006*"olan" + 0.005*"yeni" + 0.005*"oldu"')
(7, '0.010*"olarak" + 0.007*"olan" + 0.005*"ancak" + 0.005*"olduğu" + 0.005*"olduğunu" + 0.005*"dr" + 0.004*"göre"')
(8, '0.007*"hapis" + 0.007*"hakkında" + 0.006*"göz

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
# Prepare the visualization
lda_display = pyLDAvis.gensim_models.prepare(lda_model, term_matrix, word_list)

# Display the visualization within Colab
pyLDAvis.display(lda_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
# Function to find the dominant topic
def dominant_topic(ldamodel, corpus):
    # DataFrame to store the dominant topic information
    sent_topics_df = pd.DataFrame()

    # Iterate through each document in the corpus
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)  # Sort by the dominant topic's contribution

        # Get the dominant topic, percentage contribution, and keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # The first element in the sorted row is the dominant topic
                wp = ldamodel.show_topic(topic_num, topn=30)  # Get top 30 words in the topic
                topic_keywords = ", ".join([word for word, prop in wp])  # Combine the topic keywords
                # Create a new DataFrame for the new row
                new_row = pd.DataFrame([[int(topic_num), round(prop_topic, 4), topic_keywords]], columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])
                # Concatenate the new row to the existing DataFrame
                sent_topics_df = pd.concat([sent_topics_df, new_row], ignore_index=True)
            else:
                break

    return sent_topics_df

# Assuming you have your term_matrix and lda_model ready
df_dominant_topic = dominant_topic(ldamodel=lda_model, corpus=term_matrix)

# Show the first 10 rows of the result
df_dominant_topic.head(10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,0,0.5397,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
1,4,0.5807,"belediye, chp, var, başkan, parti, adayı, başk..."
2,0,0.5354,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
3,4,0.7056,"belediye, chp, var, başkan, parti, adayı, başk..."
4,0,0.9492,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
5,0,0.9007,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
6,4,0.5068,"belediye, chp, var, başkan, parti, adayı, başk..."
7,8,0.6838,"hapis, hakkında, gözaltına, yıl, ceza, tarafın..."
8,4,0.6967,"belediye, chp, var, başkan, parti, adayı, başk..."
9,5,0.3911,"olay, polis, kişi, geldi, ekipleri, meydana, g..."


In [49]:
df_dominant_topic.head(50)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,0,0.5397,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
1,4,0.5807,"belediye, chp, var, başkan, parti, adayı, başk..."
2,0,0.5354,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
3,4,0.7056,"belediye, chp, var, başkan, parti, adayı, başk..."
4,0,0.9492,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
5,0,0.9007,"yüzde, tl, bin, milyon, milyar, dolar, göre, l..."
6,4,0.5068,"belediye, chp, var, başkan, parti, adayı, başk..."
7,8,0.6838,"hapis, hakkında, gözaltına, yıl, ceza, tarafın..."
8,4,0.6967,"belediye, chp, var, başkan, parti, adayı, başk..."
9,5,0.3911,"olay, polis, kişi, geldi, ekipleri, meydana, g..."
